In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\MCW\\Documents\\Ajith\\MLOps\\project_02\\Interstate_Traffic_Volume_Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from trafficVolumePrediction.constants import *
from trafficVolumePrediction.utils.common import read_yaml, create_directories
import tensorflow as tf

[2023-12-29 21:27:51,504: WARNING: module_wrapper: From c:\Users\MCW\.conda\envs\mlops\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "data_trafficVolume.csv")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE
        )

        return training_config

In [7]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        data = pd.read_csv(self.config.training_data)
        x = np.array(data[["holiday","temp","rain_1h","snow_1h","clouds_all","weather_main","weather_description","year","month","day","hour"]])
        y = np.array(data[["traffic_volume"]])
        self.xtrain, self.xval, self.ytrain, self.yval = train_test_split(x, y, 
                                                        test_size=0.20, 
                                                        random_state=33)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = len(self.xtrain) // self.config.params_batch_size
        self.validation_steps = len(self.xval)  // self.config.params_batch_size
        print(self.steps_per_epoch)
        print(self.validation_steps)
        print(callback_list)
        self.model.fit(
            self.xtrain,
            self.ytrain,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_split=0.20,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [27]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e


[2023-12-29 22:27:07,809: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-29 22:27:07,813: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-29 22:27:07,816: INFO: common: created directory at: artifacts]
[2023-12-29 22:27:07,817: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-12-29 22:27:07,818: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-12-29 22:27:07,821: INFO: common: created directory at: artifacts\training]
77
19
[<keras.src.callbacks.TensorBoard object at 0x0000021D028DCA10>, <keras.src.callbacks.ModelCheckpoint object at 0x0000021D01C51910>]
Epoch 1/10
77/77 [==============================] - 9s 86ms/step - loss: 14556118.0000 - accuracy: 2.2690e-04 - val_loss: 14678682.0000 - val_accuracy: 0.0000e+00
Epoch 2/10
 3/77 [>.............................] - ETA: 3s - loss: 14927722.0000 - accuracy: 0.0000e+00

c:\Users\MCW\.conda\envs\mlops\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


77/77 [==============================] - 6s 80ms/step - loss: 14556119.0000 - accuracy: 2.2690e-04 - val_loss: 14678682.0000 - val_accuracy: 0.0000e+00
Epoch 3/10
77/77 [==============================] - 6s 74ms/step - loss: 14556118.0000 - accuracy: 2.2690e-04 - val_loss: 14678682.0000 - val_accuracy: 0.0000e+00
Epoch 4/10
77/77 [==============================] - 6s 72ms/step - loss: 14556115.0000 - accuracy: 2.2690e-04 - val_loss: 14678682.0000 - val_accuracy: 0.0000e+00
Epoch 5/10
77/77 [==============================] - 6s 80ms/step - loss: 14556116.0000 - accuracy: 2.2690e-04 - val_loss: 14678682.0000 - val_accuracy: 0.0000e+00
Epoch 6/10
77/77 [==============================] - 5s 71ms/step - loss: 14556117.0000 - accuracy: 2.2690e-04 - val_loss: 14678682.0000 - val_accuracy: 0.0000e+00
Epoch 7/10
77/77 [==============================] - 6s 79ms/step - loss: 14556118.0000 - accuracy: 2.2690e-04 - val_loss: 14678682.0000 - val_accuracy: 0.0000e+00
Epoch 8/10
77/77 [===============